# Preprocessing des Referenzdatensatzes

1. Anonymisierung der @user-Erwähnungen zu "@user
2. Duplikate entfernen (bei Paaren sehr ähnlicher Tweets einen entfernen)
3. Mit den verschiedenen Annotationsebenen- und detaills separat speichern

## 1. Anonymisierung der @user-Erwähnungen

In [11]:
import re

def anonym_atuser(tweet):
    tweet_anonym = re.sub('@[^@ ]+?@', '@user@', tweet)
    tweet_anonym = re.sub('@[^@ ]+? ', '@user ', tweet_anonym)
    tweet_anonym = re.sub('@[^@ ]+?$', '@user', tweet_anonym)
    return tweet_anonym

# von '@rspctfl@houelle_beck @ergroovt @ThomasMichael71 @ksemann2 @DrKassandraPari Dann hättest Du nichts dagegen, wenn ein Lehrer ein Zeichen d PKK im Unterricht trägt? Oder die Flagge Israels? Oder ein Anstecker der AfD? @Innenwelttramp'
# zu '@user@user @user @user @user @user Dann hättest Du nichts dagegen, wenn ein Lehrer ein Zeichen d PKK im Unterricht trägt? Oder die Flagge Israels? Oder ein Anstecker der AfD? @user'

## 2. Duplikate entfernen

In [2]:
# Jaccard-Index-Funktionen zur Berechnung der Ähnlichkeit

def make_bag(string1):
    """ String als Multiset speichern; 
        In Form eines dicts: jedes vorkommende Unigram als Key, die Anzahl der Nutzungen als Value
        Als zusätzlichen Wert auch die Länge des Strings speichern
    """
    stringlist = list(string1)
    stringbag = dict()
    stringset = set(string1)
    for ch1 in stringset:
        stringbag[ch1] = stringlist.count(ch1)
    stringbag["len"] = len(string1)
    return stringbag

def jaccard_multisets(bag1, bag2):
    """Den Jaccard-Index für zwei Multisets (von zwei Strings) berechnen.
       Max.-Wert: 0.5
    """

    # Schnittmenge bauen, die Gesamtlänge speichern
    schnitt_len = 0
    tok1 = set(bag1.keys()) - {"len"}
    tok2 = set(bag2.keys()) - {"len"}
    for gram in tok1:
        if gram in tok2:
            l = min(bag1[gram],bag2[gram])
            schnitt_len += l

    # Länge der Vereinigung ermitteln
    vereinigung_len = bag1["len"] + bag2["len"]
    
    jaccard_index = schnitt_len / vereinigung_len
    return jaccard_index

In [ ]:
tweet1 = "@Karl_Lauterbach Besser ein Amateur der lernfähig ist, als das verlauste Pack der abgefuckten SPD - SCHMAROTZER, PÄDOPHILE und DENUNZIANTEN !!!"
tweet2 = "@ThomasOppermann SPD - SCHMAROTZER,PÄDOPHILE UND DENUNZIANTEN   oder   SCHEINHEILGSTE PARTEI DEUTSCHLANDS !!!"
tweet3 = ""
tweet4 = "@spdde @hubertus_heil SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet5 = "SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet6 = "Deutsche Medien, Halbwahrheiten und einseitige Betrachtung, wie bei allen vom Staat finanzierten 'billigen' Propagandainstitutionen 😜"
tweet7 = "Bevor sich PL an Angies Krimigranten-Quoten beteiligen wird, verlassen sie auch die EU u. Konzerne könnten ihren Mist ins Baltikum fliegen 😜"

jind = jaccard_multisets(make_bag(tweet6), make_bag(tweet7))
print(jind)

In [1]:
def calculate_bags(corpus):
    """Multisets (bags) für alle Tweets in einem Korpus berechen.
    Input: Korpus
    Output: Liste von Multisets (Index in der Liste == Index in der Korpusliste)
    """
    bags = []
    for i in range(len(corpus)):
        bags.append(make_bag(corpus[i][1]))
    return bags

def collect_duplicates(corpus, bags):
    """ Duplikate/sehr ähnliche Tweetpaare sammeln
    Input: Korpus, Liste von Multisets für alle Tweets im Korpus
    Output: Liste potentieller Duplikate (als Tupel der Korpuseinträge)
    """
    duplicates = []
    for i in range(len(corpus)):
        #if i % 100 == 0: print("i",i)
        for j in range(len(corpus)):
            if i == j: continue
            else:
                jacc = jaccard_multisets(bags[i],bags[j])
                if jacc >= 0.47:
                    #print(jacc)
                    duplicates.append((corpus[i],corpus[j]))
    
    return duplicates

def rem_duplicates(duplicates):
    """Sortiert die Liste der Duplikate, so dass Paare von IDs nur noch einmal vorkommen.
    (Sie werden zunächst doppelt gefunden, jeweils einmal in jede Vergleichsrichtung)
    """
    dups = []
    dup_ids = []
    for i in duplicates:
        if not {i[0][0], i[1][0]} in dup_ids:
            dup_ids.append({i[0][0], i[1][0]})
            dups.append(i)
    return dups

## 3. Annotationen formatieren und speichern

- Kürzel mit Legende für die Labels erstellen

- in unterschiedlicher Detailgenauigkeit speichern:
    - VVH Ja/Nein, VVH-Allg Ja/Nein, VVH-NS Ja/Nein
    - Gruppe Ja/Nein, Welches Gruppenmerkmal/Keine Gruppe, Welche Gruppe genau/Keine Gruppe 
    - Handlung VVH-Allg Ja/Nein, Welche Handlung genau/Keine Handlung
    - Handlung VVH-NS Ja/Nein, Welche Handlung genau/Keine Handlung
